In [2]:
!conda install --yes -c pytorch pytorch=1.7.1 torchvision cudatoolkit=11.0
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git


# >>>>>>>>>>>>>>>>>>>>>> ERROR REPORT <<<<<<<<<<<<<<<<<<<<<<

    Traceback (most recent call last):
      File "/opt/conda/lib/python3.10/site-packages/conda/gateways/repodata/__init__.py", line 161, in conda_http_errors
        yield
      File "/opt/conda/lib/python3.10/site-packages/conda/gateways/repodata/__init__.py", line 127, in repodata
        response.raise_for_status()
      File "/opt/conda/lib/python3.10/site-packages/requests/models.py", line 1021, in raise_for_status
        raise HTTPError(http_error_msg, response=self)
    requests.exceptions.HTTPError: 404 Client Error: Not Found for url: https://conda.anaconda.org/rapidsai/linux-64/current_repodata.json
    
    During handling of the above exception, another exception occurred:
    
    Traceback (most recent call last):
      File "/opt/conda/lib/python3.10/site-packages/requests/models.py", line 971, in json
        return complexjson.loads(self.text, **kwargs)
      File "/opt/conda/lib/python3.10/site-packages

In [3]:
# import
import torch
import clip
import os
from torchvision.datasets import CIFAR100

In [12]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:10<00:00, 33.1MiB/s]


In [6]:
# Download the dataset
cifar100 = CIFAR100(root=os.path.expanduser("~/.cache"), download = True, train = True)

100%|██████████| 169001437/169001437 [00:10<00:00, 15764570.28it/s]


Extracting /root/.cache/cifar-100-python.tar.gz to /root/.cache


In [8]:
cifar100_data = cifar100.data
cifar100_targets = cifar100.targets

In [13]:
## information on FMNIST dataset
print('CIFAR100 data shape:', cifar100_data.shape) # array
print('CIFAR100 targets shape:', len(cifar100_targets)) # list

CIFAR100 data shape: (50000, 32, 32, 3)
CIFAR100 targets shape: 50000


In [14]:
import numpy as np

for i in zip(np.unique(cifar100_targets), cifar100.classes):
    print(i[0], ':', i[1])

0 : apple
1 : aquarium_fish
2 : baby
3 : bear
4 : beaver
5 : bed
6 : bee
7 : beetle
8 : bicycle
9 : bottle
10 : bowl
11 : boy
12 : bridge
13 : bus
14 : butterfly
15 : camel
16 : can
17 : castle
18 : caterpillar
19 : cattle
20 : chair
21 : chimpanzee
22 : clock
23 : cloud
24 : cockroach
25 : couch
26 : crab
27 : crocodile
28 : cup
29 : dinosaur
30 : dolphin
31 : elephant
32 : flatfish
33 : forest
34 : fox
35 : girl
36 : hamster
37 : house
38 : kangaroo
39 : keyboard
40 : lamp
41 : lawn_mower
42 : leopard
43 : lion
44 : lizard
45 : lobster
46 : man
47 : maple_tree
48 : motorcycle
49 : mountain
50 : mouse
51 : mushroom
52 : oak_tree
53 : orange
54 : orchid
55 : otter
56 : palm_tree
57 : pear
58 : pickup_truck
59 : pine_tree
60 : plain
61 : plate
62 : poppy
63 : porcupine
64 : possum
65 : rabbit
66 : raccoon
67 : ray
68 : road
69 : rocket
70 : rose
71 : sea
72 : seal
73 : shark
74 : shrew
75 : skunk
76 : skyscraper
77 : snail
78 : snake
79 : spider
80 : squirrel
81 : streetcar
82 : sunflow

In [15]:
total_correct = 0

for i in range(0, 50000):
    image, class_id = cifar100[i]
    image_input = preprocess(image).unsqueeze(0).to(device)
    text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in cifar100.classes]).to(device)
    
    #Calculate features
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)
    
    
    
    # Pick the top 1 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity[0].topk(1)
    
    
    if indices[0] == cifar100_targets[i]:
        total_correct += 1

print(total_correct)
    

30901


In [4]:
total_correct = 30901
accuracy = (100.0) * total_correct / 50000
print(f"분류 정확도: {accuracy:.2f}%")

분류 정확도: 61.80%
